In [32]:
import pandas as pd
import numpy as np
import xgboost as xgb

from rpy2.robjects.packages import importr
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri

In [33]:
print('Load data...')
train = pd.read_csv("./input/train.csv")
target = train['target'].values
train = train.drop(['ID','target'],axis=1)

test = pd.read_csv("./input/test.csv")
id_test = test['ID'].values
test = test.drop(['ID'],axis=1)

Load data...


In [34]:
#create new feature for each feature which is true if it is NA and false otherwise
df_naList_train = pd.DataFrame()
df_naList_test = pd.DataFrame()

for col in list(train):
    temp_train = train[col].isnull().astype(int)
    temp_test = test[col].isnull().astype(int)
    
    df_naList_train = pd.concat([df_naList_train,temp_train],axis=1)
    df_naList_test = pd.concat([df_naList_test,temp_test],axis=1)

In [36]:
#Count NAs per row
train['na_prop'] = ((train.isnull().sum(axis=1)/train.shape[1])*100).round(2)
test['na_prop'] = ((test.isnull().sum(axis=1)/test.shape[1])*100).round(2)

In [82]:
train_NACounts = pd.DataFrame(train.count(axis=0)).reset_index()
test_NACounts = pd.DataFrame(test.count(axis=0)).reset_index()

train_NACounts.columns = ['Param','NA_Count_Train']
test_NACounts.columns = ['Param','NA_Count_Test']

train_NACounts['NA_Count_Train'] = len(train)-train_NACounts['NA_Count_Train']
test_NACounts['NA_Count_Test'] = len(test)-test_NACounts['NA_Count_Test']

train_NACounts['NA_Count_Train_Perc'] = (train_NACounts['NA_Count_Train']/len(train))*100
test_NACounts['NA_Count_Test_Perc'] = (test_NACounts['NA_Count_Test']/len(test))*100

merged_NACounts = train_NACounts.merge(test_NACounts,on="Param")
merged_NACounts.sort(['NA_Count_Train_Perc'],ascending=False, inplace=True)
#77.099237
#77.862595

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:14: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [131]:
#Filter the train subset with least number of missing values per column and see what parameters are missing
train_nearComp = train[train['na_prop']<15]

In [132]:
try_ = pd.DataFrame(train_nearComp.count(axis=0)).reset_index()
try_.columns = ['Param','NA_Count_Train']
try_.sort(['NA_Count_Train'],ascending=True, inplace=True)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  app.launch_new_instance()


In [ ]:
train.v56.value_counts(dropna=False)

In [134]:
train_nearComp.na_prop.value_counts()

0.76     28488
0.00     17756
1.52     14179
2.27      1690
3.03      1377
3.79        41
4.55        33
5.30        16
6.06         5
6.82         5
7.58         4
9.85         4
12.12        4
9.09         3
13.64        3
10.61        1
8.33         1
12.88        1
Name: na_prop, dtype: int64

In [136]:
train['na_prop'].value_counts()

0.76     28488
76.52    26967
0.00     17756
77.27    15262
1.52     14179
75.76     4417
2.27      1690
3.03      1377
62.12     1049
78.03      708
18.94      570
62.88      458
61.36      457
19.70      254
78.79      216
79.55       83
20.45       47
63.64       43
3.79        41
64.39       41
81.06       35
4.55        33
81.82       26
80.30       23
49.24       19
5.30        16
48.48       11
82.58        6
6.82         5
6.06         5
50.00        5
12.12        4
9.85         4
7.58         4
9.09         3
13.64        3
21.21        2
25.00        1
8.33         1
89.39        1
66.67        1
65.91        1
10.61        1
24.24        1
51.52        1
83.33        1
21.97        1
28.79        1
12.88        1
28.03        1
65.15        1
Name: na_prop, dtype: int64